In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
os.getcwd()


'C:\\Users\\omprakash.bankol\\Desktop\\MARS_Russia\\Analytical Dataset Creation\\Code'

In [2]:
Nielsen_data = pd.read_csv(r"../Data/Raw Data/(BARS)MERGED 2017-2020 NIELSEN BARS DATA LONG FORMAT.csv")

#drop SKUxWeek duplicates
Nielsen_data=Nielsen_data.drop_duplicates(subset=['SKU','Week'])
Nielsen_data.reset_index(inplace=True,drop=True)

sku_ppg =pd.read_csv(r"../Data/MappingFile/SKU_PPG_Mapping.csv")
week_mapping = pd.read_excel(r"../Data/MappingFile/Week Mapping.xlsx", sheet_name="Mapping")

In [3]:
Nielsen_data["Nielsen_Sales"] = Nielsen_data["1:Value (in 1000 RUR)"]*1000
Nielsen_data["Nielsen_Units"] = Nielsen_data["1:Units (in 1000 PACKS)"]*1000
cols = ["Manufacturer","Brand","SKU","Week","Nielsen_Sales","Nielsen_Units"]
Nielsen_data = Nielsen_data[cols]

In [4]:
#Date
week_mapping['Week_no'] = week_mapping['Week']
Nielsen_data['Year'] = pd.to_numeric(Nielsen_data['Week'].str.slice(2,6))
Nielsen_data['Week_no'] = pd.to_numeric(Nielsen_data['Week'].str.slice(7,9))
Nielsen_data['Week_no'] = "W"+pd.to_numeric(Nielsen_data['Week_no']).astype(str)
Nielsen_data = pd.merge(Nielsen_data,week_mapping, on=["Year", "Week_no"], how='left')

In [5]:
Nielsen_data['Date'] = pd.to_datetime(Nielsen_data['Week Start date'])

In [6]:
cols = ["Manufacturer","Brand","SKU","Year","Date","Nielsen_Sales","Nielsen_Units"]

#after 2017
Nielsen_data=Nielsen_data.loc[Nielsen_data['Year'] > 2017,]
Nielsen_data.reset_index(drop=True,inplace=True)
Nielsen_data.head(5)

,Manufacturer,Brand,SKU,Week_x,Nielsen_Sales,Nielsen_Units,Year,Week_no,Week Start date,Week_y,Month_Name,Month,Date
0,ALL OTHER MANUFACTURERS,ALL OTHER BRANDS,TORERO BROWNI MCH V ROLL 0.18KG,W 2018 01,NaN,NaN,2018,W1,2017-12-31,W1,January,1,2017-12-31
1,ALL OTHER MANUFACTURERS,ALL OTHER BRANDS,TORERO BROWNI MCH V ROLL 0.18KG,W 2018 02,NaN,NaN,2018,W2,2018-01-07,W2,January,1,2018-01-07
2,ALL OTHER MANUFACTURERS,ALL OTHER BRANDS,TORERO BROWNI MCH V ROLL 0.18KG,W 2018 03,NaN,NaN,2018,W3,2018-01-14,W3,January,1,2018-01-14
3,ALL OTHER MANUFACTURERS,ALL OTHER BRANDS,TORERO BROWNI MCH V ROLL 0.18KG,W 2018 04,NaN,NaN,2018,W4,2018-01-21,W4,January,1,2018-01-21
4,ALL OTHER MANUFACTURERS,ALL OTHER BRANDS,TORERO BROWNI MCH V ROLL 0.18KG,W 2018 05,NaN,NaN,2018,W5,2018-01-28,W5,January,1,2018-01-28


In [7]:
######PPG Competitors##########
#comp_ppg = pd.read_excel(r"../Process/Comp/Nielsen_Competitors_Choco_Data_all_AS.xlsx")
comp_ppg =pd.read_excel(r"../Process/Competitor/Comp_decide/Nielsen_Competitors_Choco_Data_all_AS.xlsx")
comp_ppg.rename(columns={'Competitor_SKU':'SKU'},inplace=True)
print(comp_ppg.shape)
comp_ppg.dropna(subset=['Competitor_PPG'],inplace=True)
comp_ppg=comp_ppg.loc[(comp_ppg['Competitor_Weight_in_range']==1 )&(comp_ppg['Remarks']!='No data'), ]
comp_ppg.reset_index(inplace=True,drop=True)
comp_ppg.shape

(1238, 17)


(176, 17)

In [8]:
#competitor sku count and share
comp_ppg['Comp_SKU_count']=1
comp_ppg_sku = pd.DataFrame(comp_ppg.groupby(['PPG','Competitor_PPG'])['SKU_Sales_Share.2018','SKU_Sales_Share.2019','Comp_SKU_count'].sum())
comp_ppg_sku.head()
comp_ppg_sku.reset_index(inplace=True,drop=False)


In [9]:
#Merge 
comp_ppg_data =pd.merge(comp_ppg,Nielsen_data,on=['SKU'],how='left')
comp_ppg_data.head()

,PPG,PackSize,SKU,Competitor_Manufacturer,Competitor_Brand,Competitor_PackSize,Competitor_PPG,Competitor_Weight_in_range,Corr,NA_prct,...,Week_x,Nielsen_Sales,Nielsen_Units,Year,Week_no,Week Start date,Week_y,Month_Name,Month,Date
0,BIG BARS,"82.5, 81, 95, 82",PICNIC(MONDELEZ) BIG MCH CAR&PEANUT&PUF.RICE&R...,MONDELEZ INTERNATIONAL,PICNIC (MONDELEZ INTERNATIONAL),76.0,MONDELEZ_PICNIC,1,-0.290009,0.0,...,W 2018 01,2403100.0,46200.0,2018,W1,2017-12-31,W1,January,1,2017-12-31
1,BIG BARS,"82.5, 81, 95, 82",PICNIC(MONDELEZ) BIG MCH CAR&PEANUT&PUF.RICE&R...,MONDELEZ INTERNATIONAL,PICNIC (MONDELEZ INTERNATIONAL),76.0,MONDELEZ_PICNIC,1,-0.290009,0.0,...,W 2018 02,10416400.0,310100.0,2018,W2,2018-01-07,W2,January,1,2018-01-07
2,BIG BARS,"82.5, 81, 95, 82",PICNIC(MONDELEZ) BIG MCH CAR&PEANUT&PUF.RICE&R...,MONDELEZ INTERNATIONAL,PICNIC (MONDELEZ INTERNATIONAL),76.0,MONDELEZ_PICNIC,1,-0.290009,0.0,...,W 2018 03,10178900.0,316400.0,2018,W3,2018-01-14,W3,January,1,2018-01-14
3,BIG BARS,"82.5, 81, 95, 82",PICNIC(MONDELEZ) BIG MCH CAR&PEANUT&PUF.RICE&R...,MONDELEZ INTERNATIONAL,PICNIC (MONDELEZ INTERNATIONAL),76.0,MONDELEZ_PICNIC,1,-0.290009,0.0,...,W 2018 04,4737900.0,118100.0,2018,W4,2018-01-21,W4,January,1,2018-01-21
4,BIG BARS,"82.5, 81, 95, 82",PICNIC(MONDELEZ) BIG MCH CAR&PEANUT&PUF.RICE&R...,MONDELEZ INTERNATIONAL,PICNIC (MONDELEZ INTERNATIONAL),76.0,MONDELEZ_PICNIC,1,-0.290009,0.0,...,W 2018 05,4022000.0,89600.0,2018,W5,2018-01-28,W5,January,1,2018-01-28


In [10]:
comp_ppg_data1=comp_ppg_data.groupby(['PPG','Competitor_PPG','Date'])['Nielsen_Sales','Nielsen_Units'].sum()
comp_ppg_data1.reset_index(inplace=True,drop=False)
comp_ppg_data1.head(10)


,PPG,Competitor_PPG,Date,Nielsen_Sales,Nielsen_Units
0,BIG BARS,MARS_BIG BARS_OTHERS,2017-12-31,356800.0,7200.0
1,BIG BARS,MARS_BIG BARS_OTHERS,2018-01-07,415400.0,8400.0
2,BIG BARS,MARS_BIG BARS_OTHERS,2018-01-14,483700.0,9800.0
3,BIG BARS,MARS_BIG BARS_OTHERS,2018-01-21,800200.0,16300.0
4,BIG BARS,MARS_BIG BARS_OTHERS,2018-01-28,1206100.0,24300.0
5,BIG BARS,MARS_BIG BARS_OTHERS,2018-02-04,3887200.0,76200.0
6,BIG BARS,MARS_BIG BARS_OTHERS,2018-02-11,6147300.0,119000.0
7,BIG BARS,MARS_BIG BARS_OTHERS,2018-02-18,12924800.0,289600.0
8,BIG BARS,MARS_BIG BARS_OTHERS,2018-02-25,13394500.0,306600.0
9,BIG BARS,MARS_BIG BARS_OTHERS,2018-03-04,12727200.0,294000.0


In [11]:
#ASP
comp_ppg_data1['Competitor_Price']=(comp_ppg_data1['Nielsen_Sales'])/(comp_ppg_data1['Nielsen_Units'])
#comp_ppg_data1['Competitor_Price'] = pd.to_numeric(comp_ppg_data1['Competitor_Price'], errors='coerce')

In [12]:
#For summary
comp_ppg_data2=comp_ppg_data1.copy()
comp_ppg_data2.rename(columns={'Nielsen_Sales':'Competitor_Sales','Nielsen_Units':'Competitor_Units'},inplace=True)

Nielsen_data1 = pd.merge(Nielsen_data,sku_ppg, left_on=["SKU"], right_on=["Nielsen_SKU"],how='left')
Nielsen_data1 = Nielsen_data1[Nielsen_data1['PPG'].notna()]
Nielsen_data1.reset_index(drop=True,inplace=True)

#rolled up at PPG level
Nielsen_ppg_data=Nielsen_data1.groupby(['PPG','Date'])['Nielsen_Sales','Nielsen_Units'].sum()
Nielsen_ppg_data.reset_index(drop=False,inplace=True)
Nielsen_ppg_data['PPG_Price']=Nielsen_ppg_data['Nielsen_Sales']/Nielsen_ppg_data['Nielsen_Units']
Nielsen_ppg_data.rename(columns={'Nielsen_Sales':'PPG_Sales','Nielsen_Units':'PPG_Units'},inplace=True)
Nielsen_ppg_data.head()

,PPG,Date,PPG_Sales,PPG_Units,PPG_Price
0,BIG BARS,2017-12-31,31756700.0,654800.0,48.498320
1,BIG BARS,2018-01-07,40839000.0,844400.0,48.364519
2,BIG BARS,2018-01-14,50299700.0,1040500.0,48.341855
3,BIG BARS,2018-01-21,51028100.0,1030300.0,49.527419
4,BIG BARS,2018-01-28,49259700.0,991300.0,49.692021


In [13]:
df = pd.merge(comp_ppg_data2,Nielsen_ppg_data,on=['PPG','Date'],how='left')
df.head()

,PPG,Competitor_PPG,Date,Competitor_Sales,Competitor_Units,Competitor_Price,PPG_Sales,PPG_Units,PPG_Price
0,BIG BARS,MARS_BIG BARS_OTHERS,2017-12-31,356800.0,7200.0,49.555556,31756700.0,654800.0,48.498320
1,BIG BARS,MARS_BIG BARS_OTHERS,2018-01-07,415400.0,8400.0,49.452381,40839000.0,844400.0,48.364519
2,BIG BARS,MARS_BIG BARS_OTHERS,2018-01-14,483700.0,9800.0,49.357143,50299700.0,1040500.0,48.341855
3,BIG BARS,MARS_BIG BARS_OTHERS,2018-01-21,800200.0,16300.0,49.092025,51028100.0,1030300.0,49.527419
4,BIG BARS,MARS_BIG BARS_OTHERS,2018-01-28,1206100.0,24300.0,49.633745,49259700.0,991300.0,49.692021


In [14]:
##Corrleation PPG X Competitor PPG (our quantity vs their price)
df1=df.groupby(['PPG','Competitor_PPG'])[['PPG_Units','Competitor_Price']].corr().unstack().iloc[:,1]
df1=pd.DataFrame(df1)
df1.reset_index(inplace=True,drop=False)
df1.columns = df1.columns.get_level_values(0)
df1.rename(columns={'PPG_Units':'Correlation'},inplace=True)
df1.head()

,PPG,Competitor_PPG,Correlation
0,BIG BARS,MARS_BIG BARS_OTHERS,-0.444519
1,BIG BARS,MONDELEZ_PICNIC,-0.296260
2,BIG BARS,NESTLE_KIT KAT,-0.496644
3,BIG BARS,OTHERS_SWEET+PLUS,-0.156323
4,BIG BARS,OTHERS_ZENTIS,0.112331


In [15]:
#competitor sku count and sales share (value sales)
df1=pd.merge(df1,comp_ppg_sku,on=['PPG','Competitor_PPG'],how='left')
df1.rename(columns={'SKU_Sales_Share.2018':'Comp_Sales_Share.2018','SKU_Sales_Share.2019':'Comp_Sales_Share.2019'},inplace=True)
df1.head(50)

,PPG,Competitor_PPG,Correlation,Comp_Sales_Share.2018,Comp_Sales_Share.2019,Comp_SKU_count
0,BIG BARS,MARS_BIG BARS_OTHERS,-0.444519,0.029072,1.643852e-02,6
1,BIG BARS,MONDELEZ_PICNIC,-0.296260,0.037950,4.708877e-02,2
2,BIG BARS,NESTLE_KIT KAT,-0.496644,0.016948,1.617439e-02,1
3,BIG BARS,OTHERS_SWEET+PLUS,-0.156323,0.000000,2.573774e-04,1
4,BIG BARS,OTHERS_ZENTIS,0.112331,0.004078,6.972877e-03,2
5,BIG BARS_SNICKERSCRISPER,MARS_STD BARS_OTHERS,-0.223705,0.080578,7.478008e-02,7
6,BIG BARS_SNICKERSCRISPER,MONDELEZ_PICNIC,-0.008975,0.021343,2.572254e-02,2
7,BIG BARS_SNICKERSCRISPER,NESTLE_KIT KAT,-0.622683,0.041069,4.354983e-02,1
8,BIG BARS_SNICKERSCRISPER,NESTLE_NUTS,0.026398,0.017178,1.752293e-02,2
9,BIG BARS_SNICKERSCRISPER,OTHERS_KISMET,-0.148971,0.001166,2.297189e-04,2


In [16]:
##Export Summary file####
writer = pd.ExcelWriter(r"../Process/Competitor/Nielsen_PPG_Competitors_Summary_CHOCO.xlsx", engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df1.to_excel(writer, sheet_name='Nielsen_PPG_Correlation')
df.to_excel(writer, sheet_name='Nielsen_PPG_Data')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [17]:
#######For analytical Dataset########
#Long to wide
comp_ppg_data_wide=comp_ppg_data1.pivot_table(index=['PPG','Date'], columns='Competitor_PPG', values='Competitor_Price')
comp_ppg_data_wide.head()

Competitor_PPG       AKKOND/CHEBOKSARY  FERRERO_KINDER  KDV GRUPP  KONTI  \
PPG      Date                                                              
BIG BARS 2017-12-31                NaN             NaN        NaN    NaN   
         2018-01-07                NaN             NaN        NaN    NaN   
         2018-01-14                NaN             NaN        NaN    NaN   
         2018-01-21                NaN             NaN        NaN    NaN   
         2018-01-28                NaN             NaN        NaN    NaN   

Competitor_PPG       KONTI_SHOKOTEL  KONTI_TIMI  KONTI_VYSO V SHOKOLADE  \
PPG      Date                                                             
BIG BARS 2017-12-31             NaN         NaN                     NaN   
         2018-01-07             NaN         NaN                     NaN   
         2018-01-14             NaN         NaN                     NaN   
         2018-01-21             NaN         NaN                     NaN   
         2018-01-28             NaN         NaN                     NaN   

Competitor_PPG       MARS_BIG BARS_OTHERS  MARS_MILKYWAY_OTHERS  \
PPG      Date                                                     
BIG BARS 2017-12-31             49.555556                   NaN   
         2018-01-07             49.452381                   NaN   
         2018-01-14             49.357143                   NaN   
         2018-01-21             49.092025                   NaN   
         2018-01-28             49.633745                   NaN   

Competitor_PPG       MARS_STD BARS_OTHERS       ...        OTHERS_SPARTAK  \
PPG      Date                                   ...                         
BIG BARS 2017-12-31                   NaN       ...                   NaN   
         2018-01-07                   NaN       ...                   NaN   
         2018-01-14                   NaN       ...                   NaN   
         2018-01-21                   NaN       ...                   NaN   
         2018-01-28                   NaN       ...                   NaN   

Competitor_PPG       OTHERS_STOLICHNYE SHTUCHKI  OTHERS_SWEET+PLUS  \
PPG      Date                                                        
BIG BARS 2017-12-31                         NaN                NaN   
         2018-01-07                         NaN                NaN   
         2018-01-14                         NaN                NaN   
         2018-01-21                         NaN                NaN   
         2018-01-28                         NaN                NaN   

Competitor_PPG       OTHERS_VIT`BA  OTHERS_WAFF  OTHERS_ZENTIS  \
PPG      Date                                                    
BIG BARS 2017-12-31            NaN          NaN      59.294118   
         2018-01-07            NaN          NaN      83.200000   
         2018-01-14            NaN          NaN      77.000000   
         2018-01-21            NaN          NaN      83.000000   
         2018-01-28            NaN          NaN      83.250000   

Competitor_PPG       SLAVYANKA GK/STARIY OSKOL  \
PPG      Date                                    
BIG BARS 2017-12-31                        NaN   
         2018-01-07                        NaN   
         2018-01-14                        NaN   
         2018-01-21                        NaN   
         2018-01-28                        NaN   

Competitor_PPG       SLAVYANKA GK/STARIY OSKOL_OBYKNOVENNOE CHUDO  \
PPG      Date                                                       
BIG BARS 2017-12-31                                           NaN   
         2018-01-07                                           NaN   
         2018-01-14                                           NaN   
         2018-01-21                                           NaN   
         2018-01-28                                           NaN   

Competitor_PPG       SLAVYANKA GK/STARIY OSKOL_STEP  STORCK/GERMANY  
PPG      Date                                                        
BI

In [18]:
#rename columns,ASP=Avg Selling price
comp_ppg_data_wide.rename(columns=lambda s: s+".Nielsen_ASP",inplace=True)
comp_ppg_data_wide.reset_index(inplace=True,drop=False)
comp_ppg_data_wide.columns          

Index(['PPG', 'Date', 'AKKOND/CHEBOKSARY.Nielsen_ASP',
       'FERRERO_KINDER.Nielsen_ASP', 'KDV GRUPP.Nielsen_ASP',
       'KONTI.Nielsen_ASP', 'KONTI_SHOKOTEL.Nielsen_ASP',
       'KONTI_TIMI.Nielsen_ASP', 'KONTI_VYSO V SHOKOLADE.Nielsen_ASP',
       'MARS_BIG BARS_OTHERS.Nielsen_ASP', 'MARS_MILKYWAY_OTHERS.Nielsen_ASP',
       'MARS_STD BARS_OTHERS.Nielsen_ASP', 'MONDELEZ_ALPEN.Nielsen_ASP',
       'MONDELEZ_PICNIC.Nielsen_ASP', 'NESTLE_KIT KAT.Nielsen_ASP',
       'NESTLE_LION.Nielsen_ASP', 'NESTLE_NESQUIK.Nielsen_ASP',
       'NESTLE_NUTS.Nielsen_ASP', 'NEVSKIY KONDITER/SPB.Nielsen_ASP',
       'OBYEDINYONNYE KONDITERY_ALYONKA.Nielsen_ASP',
       'OBYEDINYONNYE KONDITERY_BABAEVSKIY.Nielsen_ASP',
       'OBYEDINYONNYE KONDITERY_KOROVKA.Nielsen_ASP',
       'OBYEDINYONNYE KONDITERY_PETERBURGSKIE TAYNY.Nielsen_ASP',
       'OBYEDINYONNYE KONDITERY_SORMOVO.Nielsen_ASP',
       'ORKLA GROUP_LAIMA.Nielsen_ASP', 'ORKLA GROUP_MARTSIPAN.Nielsen_ASP',
       'ORKLA GROUP_VANA TALLIN.Nielse

In [19]:
#remove time stamp
comp_ppg_data_wide['Date'] = pd.to_datetime(comp_ppg_data_wide['Date']).dt.date

In [20]:
comp_ppg_data_wide.to_excel(r"../Process/Competitor/Nielsen_PPG_Competitors_Data_CHOCO.xlsx",index=False)